<a href="https://colab.research.google.com/github/victori4sa/Lab-DEL/blob/main/Sistema_de_gerenciamento_de_despesas_pessoais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Sistema de gerenciamento de despesas pessoais

import os
import hashlib
import datetime
from collections import defaultdict

# Função que retorna as configurações (evita variáveis globais)
def obter_configuracoes():
    return "usuarios.txt"


def despesas_filename(usuario):
    return "%s_despesas.txt" % usuario


# Inicializa o arquivo de usuários caso ele ainda não exista
def inicializar_usuarios(users_file):
    if not os.path.exists(users_file):
        f = open(users_file, "w", encoding="utf-8")
        try:
            pass
        finally:
            f.close()


# Converte a senha para hash sha256
def hash_senha(senha):
    return hashlib.sha256(senha.encode("utf-8")).hexdigest()


# Carrega todos os usuários em um dicionário
def carregar_usuarios(users_file):
    usuarios = {}

    if not os.path.exists(users_file):
        return usuarios

    f = open(users_file, "r", encoding="utf-8")
    try:
        for linha in f:
            linha = linha.strip()
            if linha == "":
                continue
            partes = linha.split(";")
            if len(partes) < 4:
                continue
            nome = partes[0]
            usuarios[nome] = {
                "senha_hash": partes[1],
                "salario": partes[2],
                "estado": partes[3]
            }
    finally:
        f.close()

    return usuarios


# Salva todos os usuários no arquivo
def salvar_usuarios(usuarios, users_file):
    f = open(users_file, "w", encoding="utf-8")
    try:
        for nome, info in usuarios.items():
            linha = "%s;%s;%s;%s\n" % (
                nome,
                info.get("senha_hash", ""),
                info.get("salario", ""),
                info.get("estado", "")
            )
            f.write(linha)
    finally:
        f.close()


# A função cadastrar agora usa o dicionário carregado e o caminho do arquivo
def cadastrar(usuarios, users_file):
    nome = input("Nome de usuário: ").strip()
    if nome == "":
        print("Nome inválido.")
        return
    if nome in usuarios:
        print("Usuário já existe.")
        return

    senha = input("Senha: ").strip()
    if senha == "":
        print("Senha inválida.")
        return

    salario_input = input("Informe seu salário mensal (ou deixe em branco para definir depois): R$ ").strip()

    if salario_input == "":
        salario_para_gravar = ""
    else:
        try:
            salario_val = float(salario_input)
            salario_para_gravar = "%.2f" % salario_val
        except Exception:
            print("Valor de salário inválido. Cadastro cancelado.")
            return

    usuarios[nome] = {
        "senha_hash": hash_senha(senha),
        "salario": salario_para_gravar,
        "estado": ""
    }

    salvar_usuarios(usuarios, users_file)

    # criação do arquivo de despesas SEM with
    f = open(despesas_filename(nome), "w", encoding="utf-8")
    try:
        pass
    finally:
        f.close()

    print("Usuário %s cadastrado com sucesso." % nome)


# Agora o login usa a estrutura já carregada em memória
def fazer_login(usuarios):
    nome = input("Usuário: ").strip()
    senha = input("Senha: ").strip()

    if nome not in usuarios:
        print("Usuário não encontrado.")
        return None, None

    info = usuarios[nome]

    if info.get("senha_hash", "") != hash_senha(senha):
        print("Senha incorreta.")
        return None, None

    print("Login realizado com sucesso!")
    return nome, info


def atualizar_campo_usuario(usuarios, nome, campo, valor, users_file):
    if nome not in usuarios:
        return
    usuarios[nome][campo] = valor
    salvar_usuarios(usuarios, users_file)


# Adiciona uma despesa no arquivo do usuário
def adicionar_despesa(usuario):
    categoria = input("Categoria: ").strip()
    if categoria == "":
        print("Categoria inválida.")
        return

    descricao = input("Descrição (opcional): ").strip()

    valor_str = input("Valor (ex: 12.50): R$ ").strip()
    try:
        valor = float(valor_str)
    except Exception:
        print("Valor inválido. Despesa não registrada.")
        return

    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    linha = "%s;%s;%.2f;%s\n" % (ts, categoria, valor, descricao)

    arquivo = despesas_filename(usuario)

    f = open(arquivo, "a", encoding="utf-8")
    try:
        f.write(linha)
    finally:
        f.close()

    print("Despesa adicionada: %s | %s | R$ %.2f" % (ts, categoria, valor))


def carregar_despesas(usuario):
    arquivo = despesas_filename(usuario)
    despesas = []

    if not os.path.exists(arquivo):
        return despesas

    f = open(arquivo, "r", encoding="utf-8")
    try:
        for linha in f:
            linha = linha.strip()
            if linha == "":
                continue

            partes = linha.split(";")
            if len(partes) < 4:
                continue

            data = partes[0]
            categoria = partes[1]

            try:
                valor = float(partes[2])
            except Exception:
                valor = 0.0

            descricao = partes[3]

            despesas.append({
                "data": data,
                "categoria": categoria,
                "valor": valor,
                "descricao": descricao
            })
    finally:
        f.close()

    return despesas


def listar_despesas(usuario):
    despesas = carregar_despesas(usuario)
    if not despesas:
        print("Nenhuma despesa registrada.")
        return

    print("Lista de despesas do usuário %s:" % usuario)
    for d in despesas:
        print("%s | %-15s | R$ %7.2f | %s" %
              (d["data"], d["categoria"], d["valor"], d["descricao"]))


def total_geral(usuario):
    despesas = carregar_despesas(usuario)
    total = sum(d["valor"] for d in despesas)
    print("Total geral: R$ %7.2f" % total)
    return total


def total_por_categoria(usuario):
    despesas = carregar_despesas(usuario)
    totais = defaultdict(float)

    for d in despesas:
        totais[d["categoria"]] += d["valor"]

    if not totais:
        print("Nenhuma despesa registrada.")
        return {}

    for cat, val in totais.items():
        print("%s: R$ %7.2f" % (cat, val))

    return dict(totais)


def mostrar_saldo_restante(usuario, info_usuario, usuarios, users_file):
    salario_salvo = info_usuario.get("salario", "")

    if salario_salvo is None or salario_salvo.strip() == "":
        while True:
            s = input("Salário mensal não encontrado. Digite seu salário (ex: 2500.00): R$ ").strip()
            if s == "":
                print("Salário inválido. Digite um valor numérico.")
                continue
            try:
                salario_val = float(s)
                salario_para_gravar = "%.2f" % salario_val
                atualizar_campo_usuario(usuarios, usuario, "salario", salario_para_gravar, users_file)
                info_usuario["salario"] = salario_para_gravar
                print("Salário salvo.")
                break
            except Exception:
                print("Valor inválido. Digite um número válido.")

    try:
        salario_float = float(info_usuario.get("salario", "0") or 0.0)
    except Exception:
        salario_float = 0.0

    total = total_geral(usuario)

    saldo = salario_float - total

    print("Salário: R$ %7.2f" % salario_float)
    print("Saldo restante: R$ %7.2f" % saldo)
    return saldo


# Geração do relatório
# Recebe users_file para poder salvar salário se necessário
def gerar_relatorio(usuario, info_usuario, usuarios, users_file):
    despesas = carregar_despesas(usuario)
    salario_salvo = info_usuario.get("salario", "")

    # Se não houver salário salvo, pedir ao usuário
    if salario_salvo is None or salario_salvo.strip() == "":
        while True:
            s = input("Digite seu salário para calcular o saldo (ex: 2500.00): R$ ").strip()
            try:
                salario_val = float(s)
                salario_salvo = "%.2f" % salario_val

                # Atualiza o salário no arquivo de usuários
                atualizar_campo_usuario(usuarios, usuario, "salario", salario_salvo, users_file)
                print("Salário salvo.")
                break
            except Exception:
                print("Valor inválido. Tente novamente.")

    # Converte salário salvo para float
    try:
        salario_float = float(salario_salvo)
    except Exception:
        salario_float = 0.0

    # Soma total por categoria
    totais_cat = defaultdict(float)
    for d in despesas:
        totais_cat[d["categoria"]] += d["valor"]

    # Total geral
    total = sum(d["valor"] for d in despesas)

    # Saldo restante
    saldo = salario_float - total

    # Gera nome do arquivo com horário-dia-mês-ano
    agora = datetime.datetime.now().strftime("%H-%M-%S_%d-%m-%Y")
    nome_arquivo = "relatorio_%s_%s.txt" % (usuario, agora)

    # Abre o arquivo manualmente (sem usar "with")
    f = open(nome_arquivo, "w", encoding="utf-8")

    # Cabeçalho do relatório
    f.write("RELATÓRIO DE DESPESAS - Usuário: %s\n" % usuario)
    f.write("Gerado em: %s\n\n" % (agora.replace("_", " ")))

    # Lista todas as despesas registradas
    f.write("Lista de despesas:\n")
    if not despesas:
        f.write("(nenhuma despesa)\n")
    else:
        for d in despesas:
            f.write(
                "%s | %s | R$ %.2f | %s\n"
                % (d["data"], d["categoria"], d["valor"], d["descricao"])
            )

    # Totais por categoria
    f.write("\nTotais por categoria:\n")
    if not totais_cat:
        f.write("(nenhuma categoria)\n")
    else:
        for cat, val in totais_cat.items():
            f.write("%s: R$ %.2f\n" % (cat, val))

    # Totais finais
    f.write("\nTotal geral: R$ %.2f\n" % total)
    f.write("Salário informado: R$ %.2f\n" % salario_float)
    f.write("Saldo restante: R$ %.2f\n" % saldo)

    # Fecha o arquivo
    f.close()

    print("Relatório gerado: %s" % nome_arquivo)


# Função auxiliar para ler datas no formato correto
def ler_data_comparacao(prompt):
    # Esta função fica fora da função principal porque
    # você pediu para não usar função dentro de função
    while True:
        s = input(prompt).strip()
        try:
            return datetime.datetime.strptime(s, "%Y-%m-%d").date()
        except Exception:
            print("Data inválida. Use o formato YYYY-MM-DD.")


# Função auxiliar que filtra despesas entre datas
def filtrar_periodo_comparacao(lista_despesas, data_inicio, data_fim):
    # Vai percorrer todas as despesas e selecionar somente
    # as que estão dentro do intervalo desejado
    resultado = []
    for d in lista_despesas:
        try:
            data_obj = datetime.datetime.strptime(d["data"], "%Y-%m-%d %H:%M:%S").date()
        except Exception:
            continue

        if data_inicio <= data_obj <= data_fim:
            resultado.append(d)

    return resultado


# Comparação entre dois períodos
def comparar_periodos(usuario):
    # Carrega todas as despesas do usuário
    despesas = carregar_despesas(usuario)

    # Se não há despesas cadastradas, não tem comparação
    if not despesas:
        print("Nenhuma despesa registrada para comparar.")
        return

    # Leitura do primeiro período
    print("Período 1:")
    p1i = ler_data_comparacao("  Início (YYYY-MM-DD): ")
    p1f = ler_data_comparacao("  Fim   (YYYY-MM-DD): ")

    # Leitura do segundo período
    print("Período 2:")
    p2i = ler_data_comparacao("  Início (YYYY-MM-DD): ")
    p2f = ler_data_comparacao("  Fim   (YYYY-MM-DD): ")

    # Filtra as despesas dentro de cada período
    periodo1 = filtrar_periodo_comparacao(despesas, p1i, p1f)
    periodo2 = filtrar_periodo_comparacao(despesas, p2i, p2f)

    # Soma os valores
    total1 = sum(x["valor"] for x in periodo1)
    total2 = sum(x["valor"] for x in periodo2)

    # Mostra totais
    print("Total período 1: R$ %.2f" % total1)
    print("Total período 2: R$ %.2f" % total2)

    # Compara qual período gastou mais
    if total1 > total2:
        print("Você gastou MAIS no período 1.")
    elif total2 > total1:
        print("Você gastou MAIS no período 2.")
    else:
        print("Os períodos têm o mesmo valor gasto.")


# Permite alterar o salário manualmente
def alterar_salario(usuario, info_usuario, usuarios, users_file):
    while True:
        novo = input("Digite o novo salário (ex: 2500.00): R$ ").strip()
        try:
            salario_val = float(novo)
            salario_formatado = "%.2f" % salario_val

            # Atualiza salário no arquivo
            atualizar_campo_usuario(usuarios, usuario, "salario", salario_formatado, users_file)
            info_usuario["salario"] = salario_formatado

            print("Salário atualizado com sucesso!")
            return
        except Exception:
            print("Valor inválido. Tente novamente.")


# Menu inicial do sistema
def menu_inicial():
    while True:
        print("Opções:")
        print("1 - Cadastrar usuário")
        print("2 - Fazer login")
        print("3 - Sair")

        escolha = input("O que deseja? ").strip()

        if escolha in ("1", "2", "3"):
            return escolha
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")


# Menu principal do usuário logado
def menu_principal(usuario, info_usuario, usuarios, users_file):
    # Estado salvo indica onde o usuário estava antes de sair
    estado_salvo = info_usuario.get("estado", "") or ""

    if estado_salvo != "":
        print("Você saiu da última vez enquanto estava na opção: %s" % estado_salvo)
        resp = input("Deseja continuar dessa opção? (s/n): ").strip().lower()
        if resp == "s":
            cmd = estado_salvo
        else:
            cmd = ""
    else:
        cmd = ""

    while True:
        print("Opções:")
        print("1 - Adicionar despesa")
        print("2 - Listar despesas")
        print("3 - Total geral")
        print("4 - Total por categoria")
        print("5 - Saldo restante do mês")
        print("6 - Gerar relatório")
        print("7 - Selecionar período e comparar")
        print("8 - Alterar salário")
        print("9 - Sair")

        if cmd == "":
            cmd = input("O que deseja? ").strip()

        # Atualiza o estado do usuário no arquivo
        atualizar_campo_usuario(usuarios, usuario, "estado", cmd, users_file)

        if cmd == "1":
            adicionar_despesa(usuario)
        elif cmd == "2":
            listar_despesas(usuario)
        elif cmd == "3":
            total_geral(usuario)
        elif cmd == "4":
            total_por_categoria(usuario)
        elif cmd == "5":
            mostrar_saldo_restante(usuario, info_usuario, usuarios, users_file)
        elif cmd == "6":
            gerar_relatorio(usuario, info_usuario, usuarios, users_file)
        elif cmd == "7":
            comparar_periodos(usuario)
        elif cmd == "8":
            alterar_salario(usuario, info_usuario, usuarios, users_file)
        elif cmd == "9":
            atualizar_campo_usuario(usuarios, usuario, "estado", "", users_file)
            print("Saindo...")
            break
        else:
            print("Opção inválida. Digite um número de 1 a 9.")

        cmd = ""


# Função principal
# Aqui os usuários são carregados apenas uma vez
def main():
    users_file = obter_configuracoes()

    inicializar_usuarios(users_file)

    usuarios = carregar_usuarios(users_file)

    while True:
        escolha = menu_inicial()

        if escolha == "1":
            cadastrar(usuarios, users_file)

        elif escolha == "2":
            usuario, info = fazer_login(usuarios)
            if usuario is not None:
                menu_principal(usuario, info, usuarios, users_file)
                resp = input("Deseja encerrar o programa? (s/n): ").strip().lower()
                if resp == "s":
                    print("Programa encerrado. Até mais!")
                    break

        elif escolha == "3":
            print("Encerrando. Até mais!")
            break


# Executa o programa somente se este arquivo for rodado diretamente
if __name__ == "__main__":
    main()

Opções:
1 - Cadastrar usuário
2 - Fazer login
3 - Sair
O que deseja? 2
Usuário: victori4sa
Senha: victoria
Login realizado com sucesso!
Opções:
1 - Adicionar despesa
2 - Listar despesas
3 - Total geral
4 - Total por categoria
5 - Saldo restante do mês
6 - Gerar relatório
7 - Selecionar período e comparar
8 - Alterar salário
9 - Sair
O que deseja? 1
Categoria: Transporte
Descrição (opcional): 
Valor (ex: 12.50): R$ 8,55
Valor inválido. Despesa não registrada.
Opções:
1 - Adicionar despesa
2 - Listar despesas
3 - Total geral
4 - Total por categoria
5 - Saldo restante do mês
6 - Gerar relatório
7 - Selecionar período e comparar
8 - Alterar salário
9 - Sair
O que deseja? 1
Categoria: Transporte
Descrição (opcional): 
Valor (ex: 12.50): R$ 8.55
Despesa adicionada: 2025-11-18 23:39:32 | Transporte | R$ 8.55
Opções:
1 - Adicionar despesa
2 - Listar despesas
3 - Total geral
4 - Total por categoria
5 - Saldo restante do mês
6 - Gerar relatório
7 - Selecionar período e comparar
8 - Alterar salár